# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [51]:
TRAINING_DATA_FILE = 'C:/github/SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'C:/github/SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'C:/github/SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'C:/github/SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'C:/github/SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'C:/github/SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'C:/github/SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

# Read and Load Features from .txt Files into Numpy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [6]:
sparse_test_data[:5]

array([[8, 2, 1, 1],
       [8, 3, 1, 4],
       [8, 4, 1, 2],
       [8, 5, 1, 1],
       [8, 6, 1, 2]])

In [7]:
print('Nr. of rows in training file:', sparse_train_data.shape[0])
print('Nr. of rows in test file:', sparse_test_data.shape[0])

Nr. of rows in training file: 261124
Nr. of rows in test file: 118733


In [8]:
print('Nr. of emails in training file:', np.unique(sparse_train_data[:, 0]).size)

Nr. of emails in training file: 4051


In [9]:
print('Nr. of emails in test file:', np.unique(sparse_test_data[:, 0]).size)

Nr. of emails in test file: 1739


### How to create an Empty DataFrame

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
len(column_names)

2502

In [12]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5844, 5845, 5846])

In [13]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [14]:
full_train_data.head()

DOC_ID  CATEGORY  0  1  2  3  4  5  6  7  ...  2490  2491  2492  2493  \
0       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
1       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
2       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
3       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
4       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   

   2494  2495  2496  2497  2498  2499  
0     0     0     0     0     0     0  
1     0     0     0     0     0     0  
2     0     0     0     0     0     0  
3     0     0     0     0     0     0  
4     0     0     0     0     0     0  

[5 rows x 2502 columns]

In [15]:
sparse_train_data[10:15]

array([[ 0, 31,  1,  1],
       [ 0, 32,  1,  1],
       [ 0, 45,  1,  1],
       [ 0, 52,  1,  1],
       [ 0, 57,  1,  1]])

In [16]:
sparse_train_data[10][3]

1

# Create a Full Matrix from a Sparse Matrix

In [17]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe. 
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens. 
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix
    

In [18]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 30 s


In [19]:
full_train_data.head()

CATEGORY  0  1  2  3  4  5  6   7  8  ...  2490  2491  2492  2493  \
DOC_ID                                        ...                           
0              1  0  0  1  2  1  0  0   3  0  ...     0     0     0     0   
1              1  7  1  2  0  1  0  0   1  0  ...     0     0     0     0   
2              1  6  1  1  0  1  0  0   1  0  ...     0     0     0     0   
3              1  6  0  0  2  4  0  3  13  0  ...     0     0     0     0   
4              1  5  1  2  0  1  0  0   1  0  ...     0     0     0     0   

        2494  2495  2496  2497  2498  2499  
DOC_ID                                      
0          0     0     0     0     0     0  
1          0     0     0     0     0     0  
2          0     0     0     0     0     0  
3          0     0     0     0     0     0  
4          0     0     0     0     0     0  

[5 rows x 2501 columns]

# Training the Naive Bayes Model

## Calculating the Probability of Spam

In [20]:
full_train_data.CATEGORY.size

4051

In [21]:
full_train_data.CATEGORY.sum()

1248

In [22]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is', prob_spam)

Probability of spam is 0.3080720809676623


## Total Number of Words / Tokens

In [23]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

0     1     2     3     4     5     6     7     8     9     ...  2490  \
DOC_ID                                                              ...         
0          0     0     1     2     1     0     0     3     0     0  ...     0   
1          7     1     2     0     1     0     0     1     0     0  ...     0   
2          6     1     1     0     1     0     0     1     0     0  ...     0   
3          6     0     0     2     4     0     3    13     0     0  ...     0   
4          5     1     2     0     1     0     0     1     0     0  ...     0   

        2491  2492  2493  2494  2495  2496  2497  2498  2499  
DOC_ID                                                        
0          0     0     0     0     0     0     0     0     0  
1          0     0     0     0     0     0     0     0     0  
2          0     0     0     0     0     0     0     0     0  
3          0     0     0     0     0     0     0     0     0  
4          0     0     0     0     0     0     0     0     0  

[5 rows x 2500 columns]

In [24]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4051,)

In [25]:
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    181
4     43
dtype: int64

In [26]:
# total number of words
total_wc = email_lengths.sum()
total_wc

441938

## Number of Tokens in Spam & Ham Emails

In [27]:
# spam emails length
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1248,)

In [28]:
# total number of spam words
spam_wc = spam_lengths.sum()
spam_wc

183645

In [29]:
# non-spam emails
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2803,)

In [30]:
# accuracy check
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [31]:
# total number ham words
nonspam_wc = ham_lengths.sum()
nonspam_wc

258293

In [32]:
# checking for accuracy
spam_wc + nonspam_wc - total_wc

0

In [33]:
print('Average nr of words in spam emails {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average nr of words in ham emails {:.0f}'.format(nonspam_wc / ham_lengths.shape[0]))

Average nr of words in spam emails 147
Average nr of words in ham emails 92.149


## Summing the Tokens Occuring in Spam

In [34]:
full_train_features.shape # all emails

(4051, 2500)

In [35]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1] # selecting spam emails
train_spam_tokens.head() # top five rows

0     1     2     3     4     5     6     7     8     9     ...  2490  \
DOC_ID                                                              ...         
0          0     0     1     2     1     0     0     3     0     0  ...     0   
1          7     1     2     0     1     0     0     1     0     0  ...     0   
2          6     1     1     0     1     0     0     1     0     0  ...     0   
3          6     0     0     2     4     0     3    13     0     0  ...     0   
4          5     1     2     0     1     0     0     1     0     0  ...     0   

        2491  2492  2493  2494  2495  2496  2497  2498  2499  
DOC_ID                                                        
0          0     0     0     0     0     0     0     0     0  
1          0     0     0     0     0     0     0     0     0  
2          0     0     0     0     0     0     0     0     0  
3          0     0     0     0     0     0     0     0     0  
4          0     0     0     0     0     0     0     0     0  

[5 rows x 2500 columns]

In [36]:
train_spam_tokens.tail() # last five rows

0     1     2     3     4     5     6     7     8     9     ...  2490  \
DOC_ID                                                              ...         
1885       1     0     0     2     0     1     0     0     0     0  ...     0   
1887       2     0     0     0     0     0     0     0     0     0  ...     0   
1889       0     0     0     0     0     0     0     0     1     0  ...     0   
1890       2     0     0     0     1     0     0     2     0     0  ...     0   
1895       1     1     1     0     0     0     0     0     0     0  ...     0   

        2491  2492  2493  2494  2495  2496  2497  2498  2499  
DOC_ID                                                        
1885       0     0     0     0     0     0     0     0     0  
1887       0     0     0     0     0     0     0     0     0  
1889       0     0     0     0     0     0     0     0     0  
1890       0     0     0     0     0     0     0     0     0  
1895       0     0     0     0     0     0     0     0     0  

[5 rows x 2500 columns]

In [37]:
train_spam_tokens.shape

(1248, 2500)

In [38]:
# summed up all the spam emails in each column and added 1 for division to make non-zero calculation
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [39]:
summed_spam_tokens.tail()

2495    22
2496     3
2497     1
2498     7
2499     1
dtype: int64

## Summing the Tokens Occuring in Ham

In [40]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [41]:
summed_ham_tokens.shape

(2500,)

In [42]:
summed_ham_tokens.tail()

2495    15
2496    18
2497    30
2498    20
2499    39
dtype: int64

In [43]:
train_ham_tokens[2499].sum() + 1

39

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [44]:
# added 1 to fully implement the smoothing technique 2500 times
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.011706
1    0.005157
2    0.006683
3    0.010911
4    0.006511
dtype: float64

In [45]:
# probability is equals to 1
prob_tokens_spam.sum()

1.0

## P(Token | Ham) - Probability that a Token Occurs given the Email is Nonspam

In [46]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

1.0

# P(Token) - Probability that Token Occurs 

In [47]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [48]:
prob_tokens_all.sum()

1.0

# Save the Trained Model

In [50]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [52]:
sparse_test_data.shape

(118733, 4)

In [53]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

Wall time: 11.4 s


In [54]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [55]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)